In [2]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('/home/krisztina/Documents/MLMI/Advanced_ML/BayesianFlowNetworks/')
import wandb
from trainer import DiscreteBFNTrainer
wandb.init(project="bfn")
trainer = DiscreteBFNTrainer(wandb_project_name='bfn', device='cpu')


In [3]:
trainer.train(num_epochs=10)

Epoch 1/10, Batch 1/1856


/home/krisztina/Documents/MLMI/RL/.conda/lib/python3.11/site-packages/torch/autograd/__init__.py:266: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11040). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


Epoch 1/10, Loss: nan
./bfn_model_checkpoint.pth
Epoch 1/10, Batch 2/1856
Epoch 1/10, Loss: 0.5761930346488953
Epoch 1/10, Batch 3/1856
Epoch 1/10, Loss: 0.49328941106796265
Epoch 1/10, Batch 4/1856
Epoch 1/10, Loss: 0.42215219140052795
Epoch 1/10, Batch 5/1856
Epoch 1/10, Loss: 0.3588221073150635
Epoch 1/10, Batch 6/1856
Epoch 1/10, Loss: 0.31949061155319214
Epoch 1/10, Batch 7/1856
Epoch 1/10, Loss: 0.28888824582099915
Epoch 1/10, Batch 8/1856
Epoch 1/10, Loss: 0.2636065185070038
Epoch 1/10, Batch 9/1856
Epoch 1/10, Loss: 0.24114491045475006
Epoch 1/10, Batch 10/1856
Epoch 1/10, Loss: 0.22435910999774933
Epoch 1/10, Batch 11/1856
Epoch 1/10, Loss: 0.21024885773658752
Epoch 1/10, Batch 12/1856
Epoch 1/10, Loss: 0.19807936251163483
Epoch 1/10, Batch 13/1856
Epoch 1/10, Loss: 0.18810391426086426
Epoch 1/10, Batch 14/1856
Epoch 1/10, Loss: 0.17841733992099762
Epoch 1/10, Batch 15/1856
Epoch 1/10, Loss: 0.17031726241111755
Epoch 1/10, Batch 16/1856
Epoch 1/10, Loss: 0.16351468861103058
Ep

KeyboardInterrupt: 

In [ ]:
import torch
from datasets.dataset_utils import get_image_grid_from_tensor
model = trainer.bfn_model
model.eval()




In [ ]:
# Generate samples and priors
samples = model.sample(n_steps=20, device='cpu')
samples = samples.to(torch.float32)

In [ ]:
samples = samples.transpose(1, 3)
image_grid = get_image_grid_from_tensor(samples)
# Convert samples and priors to numpy arrays
image_grid = image_grid.detach().cpu().numpy()


In [ ]:
print(samples.shape, image_grid.shape)

In [ ]:
import wandb
import numpy as np
wandb.init(project="bayesian_flow")
image_grid = np.transpose(image_grid, (2, 1, 0))
print(image_grid.shape)
images = wandb.Image(image_grid, caption="MNIST - Sampled Images from BFN")
wandb.log({"image_samples": images})

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
# Image.fromarray(image_grid)
plt.imshow(image_grid)